<center><strong>中文分词</strong></center>
<center><strong>隐马尔可夫模型与维特比算法</strong></center>
<center><strong>隐马尔可夫模型的参数估计与测评</strong></center>

**HMM参数的参数估计**:   
HMM解决序列标注问题, 解决的是监督学习的问题. 也就是说我们现在有一些文本和与之对应的标注数据, 要训练一个HMM来拟合这些数据, 以便之后用这个模型进行数据标注任务, 用**极大似然估计**来估计参数:
1. 初始隐状态概率$\pi$的参数估计:   
$$\hat{\pi}_{q_i}=\frac{count(q^{1}_{i})}{count(o_1)}$$
上式指的是, 计算在第$1$时刻, 也就是文本中第一个字, $q^{1}_{i}$出现的次数占总第一个字$o_1$观测次数的比例, $q^{1}_{i}$上标1指的是第1时刻, 下标$i$指的是第$i$种标签(隐状态), $count$是的是记录次数.
2. 转移概率矩阵$A$的参数估计:   
$transition \ matrix$里面$A_{ij}$(矩阵的第i行第j列)指的是在$t$时刻实体标签为$q_i$, 而在$t+1$时刻实体标签转换到$q_j$的概率, 则转移概率矩阵的参数估计相当与一个二元模型$bigram$, 也就是把所有的标注序列中每相邻的两个实体标签分成一组, 统计他们出现的概率:
$$\hat{A}_{ij}=P(i_{t+1}= q_j | i_{t} = q_i)=\frac{count(q_i后面出现q_j的次数)}{count(q_i的次数)}$$
3. 发射概率矩阵$B$的参数估计:   
$emission \ matrix$中的$B_{jk}$(矩阵第j行第k列)指的是在$t$时刻由实体标签(隐状态)$q_j$生成汉字(观测结果)$v_k$的概率.   
$$\hat{B}_{jk}=P(o_{t}= v_k | i_{t} = q_j)=\frac{count(q_j与v_k同时出现的次数)}{count(q_j出现的次数)}$$
到此为止, 我们就可以遍历所有语料, 根据上面的方式得到模型的参数$A, \ B, \ \pi$的估计.

In [1]:
import numpy as np
import pandas as pd

# 导入并清洗数据

In [2]:
# In[导入文件并预处理]
string = r"..\练习二\第二届国际中文分词评测\icwb2-data\training\msr_training.utf8"
with open(string, encoding = "utf-8") as f:
    temp = f.read().split("\n")
    
# 去除掉前几行有的奇怪的 "“"
li = []
for line in temp:
    try:
        if line[0] == "“":
            line = line[1:]
            li.append(line)
    except:
        print("done!")
print(li[0:5]) # 展示前5个样本

done!
['  人们  常  说  生活  是  一  部  教科书  ，  而  血  与  火  的  战争  更  是  不可多得  的  教科书  ，  她  确实  是  名副其实  的  ‘  我  的  大学  ’  。', '  心  静  渐  知  春  似  海  ，  花  深  每  觉  影  生  香  。', '  吃  屎  的  东西  ，  连  一  捆  麦  也  铡  不  动  呀  ？', '  征  而  未  用  的  耕地  和  有  收益  的  土地  ，  不准  荒芜  。', '  这  首先  是  个  民族  问题  ，  民族  的  感情  问题  。']


# CRF需要的格式

In [3]:
# In[CRF需要的格式]
final = []
for line in li:
    for word in line.strip().split('  '):
        if len(word) == 1:
            final.append((word, 'S'))
        elif len(word) == 2:
            final.append((word[0], 'B'))
            final.append((word[1], 'E'))
        elif len(word) >= 3:
            final.append((word[0], 'B'))
            for i in range(1, len(word) - 1):
                final.append((word[i], 'M'))
            final.append((word[-1], 'E'))
    final.append(())      # 将被转换成pandas的None，代表换行
print(final[:100])

# 保存成表格状文件
# final = pd.DataFrame(final)
final

[('人', 'B'), ('们', 'E'), ('常', 'S'), ('说', 'S'), ('生', 'B'), ('活', 'E'), ('是', 'S'), ('一', 'S'), ('部', 'S'), ('教', 'B'), ('科', 'M'), ('书', 'E'), ('，', 'S'), ('而', 'S'), ('血', 'S'), ('与', 'S'), ('火', 'S'), ('的', 'S'), ('战', 'B'), ('争', 'E'), ('更', 'S'), ('是', 'S'), ('不', 'B'), ('可', 'M'), ('多', 'M'), ('得', 'E'), ('的', 'S'), ('教', 'B'), ('科', 'M'), ('书', 'E'), ('，', 'S'), ('她', 'S'), ('确', 'B'), ('实', 'E'), ('是', 'S'), ('名', 'B'), ('副', 'M'), ('其', 'M'), ('实', 'E'), ('的', 'S'), ('‘', 'S'), ('我', 'S'), ('的', 'S'), ('大', 'B'), ('学', 'E'), ('’', 'S'), ('。', 'S'), (), ('心', 'S'), ('静', 'S'), ('渐', 'S'), ('知', 'S'), ('春', 'S'), ('似', 'S'), ('海', 'S'), ('，', 'S'), ('花', 'S'), ('深', 'S'), ('每', 'S'), ('觉', 'S'), ('影', 'S'), ('生', 'S'), ('香', 'S'), ('。', 'S'), (), ('吃', 'S'), ('屎', 'S'), ('的', 'S'), ('东', 'B'), ('西', 'E'), ('，', 'S'), ('连', 'S'), ('一', 'S'), ('捆', 'S'), ('麦', 'S'), ('也', 'S'), ('铡', 'S'), ('不', 'S'), ('动', 'S'), ('呀', 'S'), ('？', 'S'), (), ('征', 'S'), ('而', 'S'), ('未', 'S'), ('用

[('人', 'B'),
 ('们', 'E'),
 ('常', 'S'),
 ('说', 'S'),
 ('生', 'B'),
 ('活', 'E'),
 ('是', 'S'),
 ('一', 'S'),
 ('部', 'S'),
 ('教', 'B'),
 ('科', 'M'),
 ('书', 'E'),
 ('，', 'S'),
 ('而', 'S'),
 ('血', 'S'),
 ('与', 'S'),
 ('火', 'S'),
 ('的', 'S'),
 ('战', 'B'),
 ('争', 'E'),
 ('更', 'S'),
 ('是', 'S'),
 ('不', 'B'),
 ('可', 'M'),
 ('多', 'M'),
 ('得', 'E'),
 ('的', 'S'),
 ('教', 'B'),
 ('科', 'M'),
 ('书', 'E'),
 ('，', 'S'),
 ('她', 'S'),
 ('确', 'B'),
 ('实', 'E'),
 ('是', 'S'),
 ('名', 'B'),
 ('副', 'M'),
 ('其', 'M'),
 ('实', 'E'),
 ('的', 'S'),
 ('‘', 'S'),
 ('我', 'S'),
 ('的', 'S'),
 ('大', 'B'),
 ('学', 'E'),
 ('’', 'S'),
 ('。', 'S'),
 (),
 ('心', 'S'),
 ('静', 'S'),
 ('渐', 'S'),
 ('知', 'S'),
 ('春', 'S'),
 ('似', 'S'),
 ('海', 'S'),
 ('，', 'S'),
 ('花', 'S'),
 ('深', 'S'),
 ('每', 'S'),
 ('觉', 'S'),
 ('影', 'S'),
 ('生', 'S'),
 ('香', 'S'),
 ('。', 'S'),
 (),
 ('吃', 'S'),
 ('屎', 'S'),
 ('的', 'S'),
 ('东', 'B'),
 ('西', 'E'),
 ('，', 'S'),
 ('连', 'S'),
 ('一', 'S'),
 ('捆', 'S'),
 ('麦', 'S'),
 ('也', 'S'),
 ('铡', 'S'),
 ('不', 'S'),
 (

# 计算pi

In [4]:
# In[计算start PI]
# 初始化PI
pi = {}
hiddens = "BEMS"
for hidden in hiddens:
    pi[hidden] = 0
    
# 统计频数
for line in li:
    word = line.strip().split('  ')[0]
    if len(word) == 1:
        pi['S'] = pi['S'] + 1
    else:
        pi['B'] = pi['B'] + 1
       
# 计算pi
total = 0
for hidden in hiddens:
    total += pi[hidden]
    
    
    
# 计算各个隐状态的概率，使用 MIN_FLOAT + LOG 避免下溢
from math import log
MIN_FLOAT = -3.14e100
for hidden in hiddens:
    pi[hidden] /= total
    try:
        pi[hidden] = log(pi[hidden])
    except:        
        pi[hidden] = MIN_FLOAT      
        
pi

{'B': -0.3648822540499067,
 'E': -3.14e+100,
 'M': -3.14e+100,
 'S': -1.185080377197017}

# 计算trans A

In [5]:
# In[计算trans A]
# final 有换行
N = len(hiddens) # 4
A = pd.DataFrame(np.zeros(shape=(N, N)))
A.index = 'B','E','M','S'
A.columns = 'B','E','M','S'
hidden_vec = [] # 每个句子的标注
for tup in final:
    if tup != ():
        hidden_vec.append(tup[1])
    else:
        for i in range(1, len(hidden_vec)):
            A[hidden_vec[i]][hidden_vec[i-1]] += 1 # 先列再行 
        hidden_vec = []
        
# 归一化, 每行的和为1
A = (A.T / np.sum(A, axis = 1)).T 
A = np.log(A)
# 不会报错, log(0)计算成了-inf
# A = A.replace(-np.inf, MIN_FLOAT)

# 将A转换成嵌套字典
dict_A = {'B':{}, 'E':{}, 'M':{}, 'S':{}}
for x in A.index:
    for y in A.columns:
        if A[y][x] != -np.inf:
           dict_A[x][y] = A[y][x]

<ipython-input-5-c114358bad7a>:18: RuntimeWarning: divide by zero encountered in log
  A = np.log(A)


# 计算emit B

In [6]:
# In[计算emit B]

# final去除换行,便于操作
while () in final:
    final.remove(())

word_set = set([x[0] for x in final])
M = len(word_set)
B = pd.DataFrame(np.zeros(shape=(N, M)))
B.index = 'B','E','M','S'
B.columns = word_set

for char in word_set: 
    for char_in_final, hidden in final:
        if char == char_in_final:
            B[char][hidden] += 1
    
B = (B.T / np.sum(B, axis = 1)).T 
B = np.log(B)

B = B.replace(-np.inf, MIN_FLOAT) # excel里数据要尽量保持同一个格式
# B.to_csv("Mytest.csv", encoding = "gbk") 
# 不会报错, log(0)计算成了-inf
# B = B.replace(-np.inf, MIN_FLOAT)


# 将B转换成嵌套字典
dict_B = {'B':{}, 'E':{}, 'M':{}, 'S':{}}
for x in B.index:
    for y in B.columns:
        if B[y][x] != MIN_FLOAT:
           dict_B[x][y] = B[y][x]

<ipython-input-6-d482c4faf7bd>:19: RuntimeWarning: divide by zero encountered in log
  B = np.log(B)


# 维特比算法 (分词)

In [7]:
# In[test]

start_p = pi
trans_p = dict_A
emit_p = dict_B
MIN_FLOAT = -3.14e100

PrevStatus = {
    'B': 'ES',
    'M': 'MB',
    'S': 'SE',
    'E': 'BM'
}

def viterbi(obs, states, start_p, trans_p, emit_p):
    V = [{}]  # tabular
    path = {}
    for y in states:  # init
        V[0][y] = start_p[y] + emit_p[y].get(obs[0], MIN_FLOAT)
        path[y] = [y]
    # print(path)
    

    
    for t in range(1, len(obs)):
        V.append({})
        newpath = {}
        for y in states:
            em_p = emit_p[y].get(obs[t], MIN_FLOAT)
            (prob, state) = max(
                [(V[t - 1][y0] + trans_p[y0].get(y, MIN_FLOAT) + em_p, y0) for y0 in PrevStatus[y]])
            V[t][y] = prob
            newpath[y] = path[state] + [y]
        path = newpath
        # print(path)

    (prob, state) = max((V[len(obs) - 1][y], y) for y in 'ES')

    return (prob, path[state])

sentence = "商品和服务"


print(viterbi(sentence, "BEMS", start_p, trans_p, emit_p))


def __cut(sentence):
    global emit_P
    prob, pos_list = viterbi(sentence, 'BEMS', start_p, trans_p, emit_p)
    begin, nexti = 0, 0
    # print pos_list, sentence
    for i, char in enumerate(sentence):
        pos = pos_list[i]
        if pos == 'B':
            begin = i
        elif pos == 'E':
            yield sentence[begin:i + 1]
            nexti = i + 1
        elif pos == 'S':
            yield char
            nexti = i + 1
    if nexti < len(sentence):
        yield sentence[nexti:]
print(list(__cut(sentence)))

(-30.346448208747017, ['B', 'E', 'S', 'B', 'E'])
['商品', '和', '服务']


# 打分

In [8]:
# In[打分]

import re

def to_region(segmentation: str) -> list:
    """
    将分词结果转换为区间
    :param segmentation: 商品 和 服务
    :return: [(0, 2), (2, 3), (3, 5)]
    """
    region = []
    start = 0
    for word in re.compile("\\s+").split(segmentation.strip()):
        end = start + len(word)
        region.append((start, end))
        start = end
    return region


def prf(gold: str, pred: str, dic) -> tuple:
    """
    计算P、R、F1
    :param gold: 标准答案文件，比如“商品 和 服务”
    :param pred: 分词结果文件，比如“商品 和服 务”
    :param dic: 词典
    :return: (P, R, F1, OOV_R, IV_R)
    """
    A_size, B_size, A_cap_B_size, OOV, IV, OOV_R, IV_R = 0, 0, 0, 0, 0, 0, 0
    with open(gold,encoding="utf-8") as gd, open(pred,encoding="utf-8") as pd:
        for g, p in zip(gd, pd):
            A, B = set(to_region(g)), set(to_region(p))
            A_size += len(A)
            B_size += len(B)
            A_cap_B_size += len(A & B)
            text = re.sub("\\s+", "", g)
            # for (start, end) in A:
            #     word = text[start: end]
            #     if word in dic:
            #         IV += 1
            #     else:
            #         OOV += 1

            # for (start, end) in A & B:
            #     word = text[start: end]
            #     if word in dic:
            #         IV_R += 1
            #     else:
            #         OOV_R += 1
    p, r = A_cap_B_size / B_size * 100, A_cap_B_size / A_size * 100
    return p, r, 2 * p * r / (p + r) # , OOV_R / OOV * 100, IV_R / IV * 100


import os,sys

sighan05 = "../练习二/第二届国际中文分词评测/icwb2-data/"
msr_dict = os.path.join(sighan05, 'gold', 'msr_training_words.utf8') # 字典
msr_test = os.path.join(sighan05, 'testing', 'msr_test.utf8')        # 测试集
msr_output = os.path.join(sighan05, 'testing', 'msr_output.txt')     # 预测输出
msr_gold = os.path.join(sighan05, 'gold', 'msr_test_gold.utf8')      # 标准答案



# 修改部分
msr_output1 = os.path.join(sighan05, 'testing', 'msr_output_NLP04.txt')     # 预测输出
f = open(msr_test, encoding="utf-8")
sentence = f.read()
f.close()
# print(list(__cut(sentence)))

# sentence = "我明天就乘飞机去上海。\n浙江财经大学是我的母校。\n"

with open(msr_output1, 'w', encoding="utf-8") as output:
    for word in __cut(sentence):
        if word != '\n':
            output.write(word + ' ')
        else:
            output.write('\n')


print("P:%.2f R:%.2f F1:%.2f" % prf(msr_gold, msr_output1, None))

P:33.92 R:35.92 F1:34.90
